## **Import necessary libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow	import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.optimizers import Adam

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import r2_score
from sklearn.compose import ColumnTransformer


## Load & Inspect Data

In [ ]:
dataset = pd.read_csv("admissions_data.csv")
dataset.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [ ]:
dataset.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.472000,107.192000,3.114000,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.295148,6.081868,1.143512,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [ ]:
dataset.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')

## Split into features and labels

In [ ]:
#Serial number not relevant, everything else considered relevant
features = dataset.iloc[:, 1:-1]
labels = dataset.iloc[:, -1]

In [ ]:
#split into train and test 
#no one-hot-encoding required as all data is numeric and not categorical
features_train, features_test, labels_train, labels_test = train_test_split(features, labels)

## Scale data 

In [ ]:
features_columns = features.columns

In [ ]:
ct = ColumnTransformer([("scale", StandardScaler(), features.columns)], remainder="passthrough")

In [ ]:
features_train_scaled = ct.fit_transform(features_train)

In [ ]:
features_test_scaled = ct.transform(features_test)

##Create Model

In [ ]:
def make_model():
  model = Sequential(name="model")
  input = InputLayer(input_shape=(features_train.shape[1]))
  model.add(input)
  model.add(layers.Dense(64, activation="relu"))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(64, activation="relu"))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(64, activation="relu"))
  model.add(layers.Dense(1))
  opt = Adam(learning_rate=0.01)
  model.compile(loss="mse", metrics="mae", optimizer=opt)
  return model

### Early Stopping

In [ ]:
es = EarlyStopping(monitor="val_loss", mode="min", patience=40)

## Fit and Evaluate

In [ ]:
model  = make_model()
model.fit(features_train_scaled, labels_train, epochs=100, batch_size=4, validation_split=0.2, verbose=0, callbacks=[es])

In [ ]:
mse, mae = model.evaluate(features_test_scaled, labels_test)

4/4 [==============================] - 0s 2ms/step - loss: 0.0039 - mae: 0.0483


In [ ]:
print(mse, mae)

0.003947214689105749 0.04829179123044014


In [ ]:
plt.plot(model.history["mae"], label="training")
plt.plot(model.history["val_mae"], label="validation")
plt.title("Model MAE")
plt.xlabel("Epochs")
plt.ylabel("MAE")

AttributeError: ignored

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from scipy.stats import randint as sp_randint

In [ ]:
param_grid = {"batch_size": sp_randint(2,16), "nb_epoch": sp_randint(10,100)}

In [ ]:
model = KerasRegressor(build_fn=make_model)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, scoring=make_scorer(mean_squared_error, greater_is_better=False), n_iter=20)

In [ ]:
grid_result = grid.fit(features_train_scaled, labels_train, verbose=0)
print(grid_result)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f8a29f5d2b0>,
                   iid='deprecated', n_iter=20, n_jobs=None,
                   param_distributions={'batch_size': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f8a279da438>,
                                        'nb_epoch': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f8a279da898>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False,
                   scoring=make_scorer(mean_squared_error, greater_is_better=False),
                   verbose=0)


In [ ]:
print(grid_result.best_score_, grid_result.best_params_)

-0.010236461047456819 {'batch_size': 3, 'nb_epoch': 71}
